In [ ]:
# | default_exp utils.grad_check
# | hide
import nbdev

nbdev.nbdev_export()
from nbdev.showdoc import *

### Tidy Utils


In [ ]:
# | export
from tidygrad.tensor import Tensor
from tidygrad.functional import relu
import numpy as np

In [ ]:
# | export
def grad_check(func, inputs, params: tuple = (), eps=1e-5, n=1000):
    for p in reversed(params):
        # Reshape to 1D so it's easier to sample random indices

        data_view = p.data.reshape(-1)  # This does not make a copy
        grad_view = p.grad.reshape(-1)

        slow_grad = np.zeros_like(grad_view)
        slow_grad_view = slow_grad.reshape(-1)

        indices = np.random.choice(np.arange(grad_view.size), size=min(n, grad_view.size), replace=False)

        indices = list(filter(lambda idx: abs(grad_view[idx]) > 1e-9, indices))  # XXX?
        for idx in indices:
            old_val = data_view[idx]

            loss = func(inputs, params)

            data_view[idx] = old_val + eps
            loss_plus_h = func(inputs, params)

            slow_grad_view[idx] = (loss_plus_h.data - loss.data) / eps

            # print(f"{idx}: loss_plus_h: {loss_plus_h.data}, loss: {loss.data}, diff: {loss_plus_h.data - loss.data}, grad: {grad_view[idx]}, slow_grad: {slow_grad_view[idx]}")
            data_view[idx] = old_val

        max_grad_diff = np.max(np.abs((slow_grad_view[indices] - grad_view[indices]) / np.maximum(np.abs(slow_grad_view[indices]), np.abs(grad_view[indices]))))

        print(f"Max gradient difference for {p.name}: {max_grad_diff*100:.4f}%")
        if max_grad_diff > 1e-2:
            raise ValueError(f"Gradient check failed for {p.name}: Max error: {max_grad_diff*100:.4f}")

In [ ]:
from tidygrad.tensor import Tensor
from tidygrad.functional import BCE_loss
from lovely_numpy import Lo

In [ ]:
Lo((np.random.randn(32, 28 * 28) @ (np.random.randn(28 * 28, 100) * 0.1) + np.random.randn(100)) @ (np.random.randn(100, 10) * 0.1))

array[32, 10] n=320 (2.5Kb) x∈[-7.627, 7.782] μ=-0.279 σ=2.947

In [ ]:
x = Tensor(np.random.randn(32, 28 * 28), "X")
# Create a 1-hot encoded tensor with 1 random 1
y = np.zeros((32, 10))
y[np.arange(32), np.random.choice(10, 32)] = 1
y = Tensor(y, "y")

w1 = Tensor(np.random.randn(28 * 28, 100) * 0.1, "w1")
b1 = Tensor(np.random.randn(100), "b1")
w2 = Tensor(np.random.randn(100, 10) * 0.1, "w2")

def NN(inputs, params: tuple, debug=list()):
    x, y = inputs
    w1, b1, w2 = params
    z1 = x.mmul(w1, "tmp").add(b1, "z1")
    a1 = relu(z1)
    z2 = a1.mmul(w2)

    loss = -BCE_loss(z2, y).sum("loss")

    debug.append((z1, a1, z2, loss))

    return loss

debug = []
loss = NN(inputs=(x, y), params=(w1, b1, w2), debug=debug)

loss.backward()

grad_check(NN, (x, y), (w1, b1, w2))

Max gradient difference for w2: 0.7700%
Max gradient difference for b1: 0.0019%
Max gradient difference for w1: 0.3036%
